In [8]:
from sunpy.net import Fido
import sunpy.net.attrs as a
import sunpy_soar
from IPython.display import display

# Solar Orbiter 8 Data Workshop Data Analysis Tutorial | Download MAG Data

## References

- [Solar Orbiter Archive](http://soar.esac.esa.int/soar/)
- [Solar Orbiter Archive / Programmatic  Access / Data Requests](http://soar.esac.esa.int/soar/#aio)
- [Solar Orbiter SOC Public / SOAR (Solar Orbiter Archive) - How to Use](https://issues.cosmos.esa.int/solarorbiterwiki/display/SOSP/SOAR+%28Solar+Orbiter+Archive%29+-+How+to+Use)
- [SunPy](https://sunpy.org/)
- [sunpy-soar](https://github.com/sunpy/sunpy-soar)

## MAG Data Products introduction

_Data products_ for Solar Orbiter can be found on http://soar.esac.esa.int/soar/. For MAG each  data product has an associated identifier with the structure

```
solo_L<level>_<name>
```

where `level` is a value in `{0,1,2,3}`, `date` has the format `YYYYMMDD`. The `name` for each data product is lower case and starts with an instrument abbreviation, in this case `mag` followed by additional data product identifiers, all lower case, each separated by a hyphen-minus. Some examples of data product identifiers are

```
solo_L0_mag-obs-normal
solo_L2_mag-rtn-burst
```

Please see section _MAG Data Products_ for a full list of all science quality data product identifiers. For MAG there is one _data item_ per day for each data product provided there is sufficient MAG telemetry for that day to construct that data product. MAG science quality data items _identifiers_ all share the same structure

```
solo_L<level>_<name>_<date>
```

where `date` has the format `YYYYMMDD`. Some examples of data items are

```
solo_L0_mag-obs-normal_20211127
solo_L2_mag-rtn-burst_20211212
```

Each data item can have multiple versions. A specific file will be associated with each version. The name of each file is the data item identifier along with the version number and file type extension. All MAG data files are CDF files with extension `cdf`

```
solo_L<level>_<name>_<date>_V<version>.cdf
```

Some example of data files are

```
solo_L2_mag-rtn-normal_20211127_V01.cdf
solo_L2_mag-rtn-normal-1-minute_20210305_V02.cdf
```

## MAG Data Product identifiers

### L0 Data Product identifiers

- `solo_L0_mag-ibs-burst`
- `solo_L0_mag-ibs-normal`
- `solo_L0_mag-obs-burst`
- `solo_L0_mag-obs-normal`
- `solo_L0_mag-pri-1920`

### L1 Data Products identifiers

- `solo_L1_mag-ibs-burst`
- `solo_L1_mag-ibs-normal`
- `solo_L1_mag-obs-burst`
- `solo_L1_mag-obs-normal`
- `solo_L1_mag-pri-1920`

### L2 Data Products identifiers

- `solo_L2_mag-gse-burst`
- `solo_L2_mag-gse-normal`
- `solo_L2_mag-gse-normal-1-minute`
- `solo_L2_mag-rtn-burst`
- `solo_L2_mag-rtn-normal`
- `solo_L2_mag-rtn-normal-1-minute`
- `solo_L2_mag-srf-burst`
- `solo_L2_mag-srf-normal`
- `solo_L2_mag-vso-normal`
- `solo_L2_mag-vso-normal-1-minute`

## Download MAG Data using the SOAR Web API

The SOAR provides a Web API through which data files can be requested and downloaded. Any program that can normally make HTTP requests can be used access the SOAR through the Web API. Please see [Solar Orbiter Archive / Programmatic  Access / Data Requests](http://soar.esac.esa.int/soar/#aio) for more details. Here is an example of downloading the latest version of MAG data item `solo_L2_mag-rtn-normal_20211127` using `wget`

In [4]:
%%sh

# Define the name of the data  item we want to download and where
DATA_ITEM="solo_L2_mag-rtn-normal_20211127"
DATA_DIR="${PWD}/data"

# Create the directory where data will be download if it doesn't exist
mkdir -p "${DATA_DIR}"

# Make a web request to receive the data product in this case using wget
API_URL_TEMPLATE="http://soar.esac.esa.int/soar-sl-tap/data?retrieval_type=LAST_PRODUCT&product_type=SCIENCE&data_item_id="
wget -O "${DATA_DIR}/${DATA_ITEM}.cdf" "${API_URL_TEMPLATE}${DATA_ITEM}"

--2022-08-21 17:22:11--  http://soar.esac.esa.int/soar-sl-tap/data?retrieval_type=LAST_PRODUCT&product_type=SCIENCE&data_item_id=solo_L2_mag-rtn-normal_20211127
Resolving soar.esac.esa.int (soar.esac.esa.int)... 193.147.153.149, 131.176.243.100, 131.176.166.81, ...
Connecting to soar.esac.esa.int (soar.esac.esa.int)|193.147.153.149|:80... connected.
HTTP request sent, awaiting response... 200 
Length: 10686693 (10M) [application/octet-stream]
Saving to: ‘/home/efauchon/projects/solo8_tutorials/MAG_tutorial/data/solo_L2_mag-rtn-normal_20211127.cdf’

     0K .......... .......... .......... .......... ..........  0%  632K 16s
    50K .......... .......... .......... .......... ..........  0%  636K 16s
   100K .......... .......... .......... .......... ..........  1% 1.16M 14s
   150K .......... .......... .......... .......... ..........  1% 59.7M 10s
   200K .......... .......... .......... .......... ..........  2% 1.25M 10s
   250K .......... .......... .......... .......... ........

## Download MAG Data using SunPy

[SunPy](https://sunpy.org/) used with the [sunpy-soar](https://github.com/sunpy/sunpy-soar) plugin provides a user friendly interface for searching the SOAR for data files and downloading. Internally this makes calls to the SOAR Web API. Please see [sunpy-soar / Example usage](https://github.com/sunpy/sunpy-soar#example-usage) for more details of how to use SunPy with sunpy-soar. Note that when using the `Product` search attribute, you should provide a string value as an argument that should be the _name_ of the data product in all capitals. Here is an example searching for and downloading the same file we previously requested using wget

In [9]:

# Prepare search for MAG data files
instrument = a.Instrument('MAG')
time = a.Time('2021-11-27', '2021-11-27')
level = a.Level(2)
product = a.soar.Product('MAG-RTN-NORMAL')

# Perform search for MAG data files
result = Fido.search(instrument & time & level & product)
display(result)

Instrument,Data product,Level,Start time,End time,Data item ID,Filename,Filesize
,,,,,,,Mbyte
str3,str14,str2,str23,str23,str31,str39,float64
MAG,MAG-RTN-NORMAL,L2,2021-11-27 00:00:00.000,2021-11-28 00:00:00.000,solo_L2_mag-rtn-normal_20211127,solo_L2_mag-rtn-normal_20211127_V01.cdf,10.687


In [3]:
# Download files
files = Fido.fetch(result, path='data', overwrite=True, progress=True)
display(files)

Files Downloaded: 100%|██████████| 1/1 [08:56<00:00, 536.60s/file]
